In [1]:
import pandas as pd
import geopandas as gp
import pdv_functions as pdv

In [2]:
#Load in clean SOS file
elections_2022_raw = pd.read_excel("./raw-from-source/statewide-races-by-precinct.xlsx",sheet_name="Master")

In [3]:
elections_2022_raw

,"November 8, 2022 General Election Official Canvass\nAll Member of the State Board of Education races are non-partisan.\n*Write-in candidates will be displayed with a (WI) designation and not party affiliation.\n*Precinct-level data is not available for write-in candidates. To view results for write-in candidates, download the Summary-level spreadsheet.",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Governor and Lieutenant Governor\n,Unnamed: 9,...,Member of the State Board of Education - District 02\n,Unnamed: 321,Member of the State Board of Education - District 03\n,Member of the State Board of Education - District 04\n,Unnamed: 324,Member of the State Board of Education - District 09\n,Unnamed: 326,Member of the State Board of Education - District 10\n,Unnamed: 328,Unnamed: 329
0,County Name,Precinct Name,Precinct Code,Region Name,Media Market,Registered Voters,Ballots Counted,Official Voter Turnout,Mike DeWine and Jon Husted (R),Timothy Grady and Dayna Bickley (WI)*,...,Teresa Fedor,Sarah McGervey,Charlotte McGuire,Katie Hofmann,Jenny Kilgore,Robert R. Fulton,John P. Hagan,Tom Jackson,Tim Miller,Cierra Lynch Shehorn
1,Total,NaN,NaN,NaN,NaN,8029950,4201368,0.523212,2580424,0,...,157401,123927,222647,182009,149472,120462,185080,136986,85316,86394
2,Percentage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.625419,0,...,0.559493,0.440507,1,0.549078,0.450922,0.394257,0.605743,0.443757,0.276375,0.279868
3,Adams,BRATTON TOWNSHIP,AAA,Southwest,Cincinnati,988,573,0.57996,491,0,...,0,0,0,0,0,0,0,0,0,0
4,Adams,BRUSH CREEK TOWNSHIP,AAB,Southwest,Cincinnati,768,367,0.477865,306,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8931,Wyandot,RICHLAND TS,ABF,Northwest,Toledo,578,354,0.612457,301,0,...,0,0,0,0,0,0,0,0,0,0
8932,Wyandot,RIDGE TS,ABH,Northwest,Toledo,368,219,0.595109,180,0,...,0,0,0,0,0,0,0,0,0,0
8933,Wyandot,SALEM TS,ABI,Northwest,Toledo,680,427,0.627941,340,0,...,0,0,0,0,0,0,0,0,0,0
8934,Wyandot,SYCAMORE TS,ABL,Northwest,Toledo,1021,574,0.562194,464,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
def clean_oh_import(dataframe):
    
    corrected_col_names = []
    for index in range(0,len(dataframe.columns)):
        col_name = dataframe.columns[index]
        col_name = col_name.replace("\n"," ")
        if "Unnamed" in col_name:
            corrected_col_names.append(corrected_col_names[index-1])
        else:
            corrected_col_names.append(col_name)
    dataframe.columns = corrected_col_names
    
    corrected_col_names = []
    for index in range(0,len(dataframe.columns)):
        col = dataframe.columns[index]
        first_row = dataframe.iloc[0,index]
        if (index > 7):
            new_column_name = col + " " + first_row 
            corrected_col_names.append(new_column_name)
        else:
            corrected_col_names.append(first_row)
    dataframe.columns = corrected_col_names
    # Note I had this as 3
    dataframe = dataframe.iloc[3:,]
    return dataframe

In [5]:
elections_2022 = clean_oh_import(elections_2022_raw)

In [6]:
races_to_clean = [i for i in elections_2022.columns if i not in ['County Name', 'Precinct Name', 'Precinct Code', 'Region Name',
       'Media Market', 'Registered Voters', 'Ballots Counted',
       'Official Voter Turnout'] and "Member of the State Board of Education" not in i and "Judge of the Court of Appeals" not in i]

In [7]:
elections_2022 = elections_2022[['County Name', 'Precinct Name', 'Precinct Code', 'Region Name',
       'Media Market', 'Registered Voters', 'Ballots Counted',
       'Official Voter Turnout'] + races_to_clean]

In [8]:
elections_2022

,County Name,Precinct Name,Precinct Code,Region Name,Media Market,Registered Voters,Ballots Counted,Official Voter Turnout,Governor and Lieutenant Governor Mike DeWine and Jon Husted (R),Governor and Lieutenant Governor Timothy Grady and Dayna Bickley (WI)*,...,State Representative - District 94 Jay Edwards (R),State Representative - District 95 Don Jones (R),State Representative - District 95 William D. Ryan (D),State Representative - District 96 Charlie DiPalma (D),State Representative - District 96 Ron Ferguson (R),State Representative - District 97 Adam Holmes (R),State Representative - District 98 Annie Homstad (WI)*,State Representative - District 98 Darrell D. Kick (R),State Representative - District 99 Sarah Fowler Arthur (R),State Representative - District 99 Kathy Zappitello (D)
3,Adams,BRATTON TOWNSHIP,AAA,Southwest,Cincinnati,988,573,0.57996,491,0,...,0,0,0,0,0,0,0,0,0,0
4,Adams,BRUSH CREEK TOWNSHIP,AAB,Southwest,Cincinnati,768,367,0.477865,306,0,...,0,0,0,0,0,0,0,0,0,0
5,Adams,LOCUST GROVE,AAD,Southwest,Cincinnati,776,424,0.546392,366,0,...,0,0,0,0,0,0,0,0,0,0
6,Adams,GREEN TOWNSHIP,AAE,Southwest,Cincinnati,381,197,0.51706,148,0,...,0,0,0,0,0,0,0,0,0,0
7,Adams,JEFFERSON TOWNSHIP,AAG,Southwest,Cincinnati,496,244,0.491935,191,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8931,Wyandot,RICHLAND TS,ABF,Northwest,Toledo,578,354,0.612457,301,0,...,0,0,0,0,0,0,0,0,0,0
8932,Wyandot,RIDGE TS,ABH,Northwest,Toledo,368,219,0.595109,180,0,...,0,0,0,0,0,0,0,0,0,0
8933,Wyandot,SALEM TS,ABI,Northwest,Toledo,680,427,0.627941,340,0,...,0,0,0,0,0,0,0,0,0,0
8934,Wyandot,SYCAMORE TS,ABL,Northwest,Toledo,1021,574,0.562194,464,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
fips_file = pd.read_csv("./raw-from-source/FIPS/US_FIPS_Codes.csv")
fips_file = fips_file[fips_file["State"]=="Ohio"]
fips_file["FIPS County"] = fips_file["FIPS County"].astype(str)
fips_file["FIPS County"] = fips_file["FIPS County"].str.zfill(3)
fips_file["County Name"] = fips_file["County Name"].replace("De Kalb","DeKalb")
fips_dict = dict(zip(fips_file['County Name'], fips_file['FIPS County']))
elections_2022['county_fips'] = elections_2022['County Name'].map(fips_dict).fillna(elections_2022['County Name'])
elections_2022['county_fips'] = elections_2022['county_fips'].astype(str)
elections_2022['county_fips'] = elections_2022['county_fips'].str.zfill(3)

# Print statements to check the county FIPs we've added
print(elections_2022['county_fips'].unique())

['001' '003' '005' '007' '009' '011' '013' '015' '017' '019' '021' '023'
 '025' '027' '029' '031' '033' '035' '037' '039' '041' '043' '045' '047'
 '049' '051' '053' '055' '057' '059' '061' '063' '065' '067' '069' '071'
 '073' '075' '077' '079' '081' '083' '085' '087' '089' '091' '093' '095'
 '097' '099' '101' '103' '105' '107' '109' '111' '113' '115' '117' '119'
 '121' '123' '125' '127' '129' '131' '133' '135' '137' '139' '141' '143'
 '145' '147' '149' '151' '153' '155' '157' '159' '161' '163' '165' '167'
 '169' '171' '173' '175']


In [10]:
races_to_clean

['Governor and Lieutenant Governor  Mike DeWine and Jon Husted (R)',
 'Governor and Lieutenant Governor  Timothy Grady and Dayna Bickley (WI)*',
 'Governor and Lieutenant Governor  Craig Patton and Collin Cook (WI)*',
 'Governor and Lieutenant Governor  Renea Turner and Adina Pelletier (WI)*',
 'Governor and Lieutenant Governor  Marshall Usher and Shannon  Walker (WI)*',
 'Governor and Lieutenant Governor  Nan Whaley and Cheryl L. Stephens (D)',
 'Attorney General  Jeffrey A. Crossman (D)',
 'Attorney General  Dave Yost (R)',
 'Auditor of State  Keith Faber (R)',
 'Auditor of State  Taylor Sappington (D)',
 'Secretary of State  Chelsea Clark (D)',
 'Secretary of State  Frank LaRose (R)',
 'Secretary of State  Terpsehore Tore Maras',
 'Treasurer of State  Scott Schertzer (D)',
 'Treasurer of State  Robert Sprague (R)',
 'Chief Justice of the Supreme Court Term Commencing 01/01/2023 Jennifer Brunner (D)',
 'Chief Justice of the Supreme Court Term Commencing 01/01/2023 Sharon L. Kennedy (

In [11]:
def get_race(race_string):
    race_string = race_string.title()
    race_string = race_string.replace("(Vote For 1)","")
    if "U.S. House" in race_string or 'Us House' in race_string or "Representative To Congress" in race_string:
        return "CON"
    elif "State House" in race_string or "State Representative" in race_string:
        return "SL"
    elif "State Senate" in race_string or "State Senator" in race_string:
        return "SU"
    elif "President" in race_string:
        return "PRE"
    elif "US Senate" in race_string or "Us Senate" in race_string or "U.S. Senator" in race_string:
        return "USS"
    elif "Public Service" in race_string:
        return "PSC"
    elif "Attorney General" in race_string:
        return "ATG"
    elif "Auditor General" in race_string or "Auditor" in race_string:
        return "AUD"
    elif "Treasurer" in race_string:
        return "TRE"
    elif "Superintendent" in race_string:
        return "SUP"
    elif "Secretary Of State" in race_string:
        return "SOS"
    elif "Governor" in race_string:
        return "GOV"
    elif "Commissioner Of Labor" in race_string:
        return "LAB"
    elif "Commissioner Of Agriculture" in race_string:
        return "AGR"
    elif "Commissioner Of Insurance" in race_string:
        return "AGR"
    elif "Chief Justice Of The Supreme Court" in race_string:
        return "CJU"
    elif "Justice Of The Supreme Court" in race_string:
        return "JUS"
    elif "Amendment" in race_string:
        if "1" in race_string:
            return "A01"
        elif "2" in race_string:
            return "A02"
        else:
            print("No race for:", race_string)
            raise ValueError
    elif "Referendum" in race_string:
        if "1" in race_string or "A" in race_string:
            return "RFA"
        elif "2" in race_string or "B" in race_string:
            return "RFB"
        else:
            print("No race for:", race_string)
            raise ValueError
    else:
        print("No race for:", race_string)
        raise ValueError
        
def get_election_type(race_string):
    if "(runoff)" in race_string:
        return "R"
    else:
        return "G"
        
def get_party(race_string):
    if "(R)" in race_string:
        return "R"
    elif "(D)" in race_string or "(Democrat)" in race_string:
        return "D"
    elif "(L)" in race_string or "(L)" in race_string:
        return "L"
    elif "(WI)" in race_string:
        return "O"
    elif race_string[0:3]=="Yes":
        return "YES"
    elif race_string[0:2]=="No":
        return "NO"
    else:
        return "O"
           
def get_name(name_string):
    if " (" in name_string and "Amendment" not in name_string and "Referendum" not in name_string:
        #print(name_string)
        name_string = name_string.split(" (")[0]
        name_string = name_string.replace("'","")
        likely_last = name_string.split(" ")[-1]
        proposed_last = likely_last[:3]
        if proposed_last in ['II', 'III', 'Jr', 'Jr.', 'Sr.', 'JR.', "JR", "IV"]:
            likely_last = name_string.split(" ")[-2]
            proposed_last = likely_last[:3]
        #print(proposed_last.upper())
        return proposed_last.upper()
    else:
        return name_string.split(" ")[-1][:3].upper()
#     name_string = name_string.split("-:-")[1]
#     name_string = name_string.replace(" (I)","")
#     name_string = name_string.replace("'","")
#     likely_last = name_string.split(" ")[-1]
#     proposed_last = likely_last[:3]
#     if proposed_last in ['II', 'III', 'Jr', 'Jr.', 'Sr.', 'JR.', "JR", "IV"]:
#         likely_last = name_string.split(" ")[-2]
#         proposed_last = likely_last[:3]
#     return proposed_last.upper()

def get_district(race_string, fill_level):
    race_string = race_string.replace(" (Vote For 1)","")
    break_word = "District "
    temp = race_string.split(break_word)[1].split(" ")[0]
    return temp.zfill(fill_level)

def column_rename_function(name_string):
    election_type = get_election_type(name_string)
    year = "22"
    party = get_party(name_string)
    race = get_race(name_string)
    district = ""
    if race in ["CON", "SU"]:
        district = get_district(name_string, 2)
        year = ""
    elif race in ["SL"]:
        district = get_district(name_string, 3)
        year = ""
    name = get_name(name_string)
    new_col_name = election_type + year + race + district + party + name
#     print("election_type: ", election_type)
#     print("year: ", year)
#     print("race: ", race)
#     print("district: ", district)
#     print("party: ", party)
#     print("name: ", name)
    if len(new_col_name) > 10:
        print(name_string)
        print(new_col_name)
    return new_col_name

# Make a dictionary that points to the new column names and checks for duplicates

update_dict = {'G22GOVRHUS':'G22GOVRDEW',
 'G22GOVDSTE':'G22GOVDWHA'}

race_updates_dict = {}
race_updates_reversed = {}
clean_dups = {}
new_names = []
for val in races_to_clean:
    new_name = column_rename_function(val)
    if new_name in update_dict.keys():
        new_name = update_dict[new_name]

    race_updates_dict[val] = new_name
    if new_name not in new_names:
        new_names.append(new_name)
        race_updates_reversed[new_name] = val
    else:
        print("Duplicate", new_name)
        print(race_updates_reversed[new_name])
        print(val)
        clean_dups[val] = race_updates_reversed[new_name]

In [12]:
elections_2022.rename(columns = race_updates_dict, inplace = True)

In [13]:
elections_2022.reset_index(inplace = True, drop = True)

In [14]:
elections_2022

,County Name,Precinct Name,Precinct Code,Region Name,Media Market,Registered Voters,Ballots Counted,Official Voter Turnout,G22GOVRDEW,G22GOVOBIC,...,GSL095RJON,GSL095DRYA,GSL096DDIP,GSL096RFER,GSL097RHOL,GSL098OHOM,GSL098RKIC,GSL099RART,GSL099DZAP,county_fips
0,Adams,BRATTON TOWNSHIP,AAA,Southwest,Cincinnati,988,573,0.57996,491,0,...,0,0,0,0,0,0,0,0,0,001
1,Adams,BRUSH CREEK TOWNSHIP,AAB,Southwest,Cincinnati,768,367,0.477865,306,0,...,0,0,0,0,0,0,0,0,0,001
2,Adams,LOCUST GROVE,AAD,Southwest,Cincinnati,776,424,0.546392,366,0,...,0,0,0,0,0,0,0,0,0,001
3,Adams,GREEN TOWNSHIP,AAE,Southwest,Cincinnati,381,197,0.51706,148,0,...,0,0,0,0,0,0,0,0,0,001
4,Adams,JEFFERSON TOWNSHIP,AAG,Southwest,Cincinnati,496,244,0.491935,191,0,...,0,0,0,0,0,0,0,0,0,001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8928,Wyandot,RICHLAND TS,ABF,Northwest,Toledo,578,354,0.612457,301,0,...,0,0,0,0,0,0,0,0,0,175
8929,Wyandot,RIDGE TS,ABH,Northwest,Toledo,368,219,0.595109,180,0,...,0,0,0,0,0,0,0,0,0,175
8930,Wyandot,SALEM TS,ABI,Northwest,Toledo,680,427,0.627941,340,0,...,0,0,0,0,0,0,0,0,0,175
8931,Wyandot,SYCAMORE TS,ABL,Northwest,Toledo,1021,574,0.562194,464,0,...,0,0,0,0,0,0,0,0,0,175


In [15]:
# Clean governor names

# Check Countywide / Statewide Totals

In [16]:
#Load in clean SOS file
elections_2022_totals = pd.read_excel("./raw-from-source/statewide-races-summary.xlsx",sheet_name="Master")

In [17]:
def clean_oh_import(dataframe):
    
    corrected_col_names = []
    for index in range(0,len(dataframe.columns)):
        col_name = dataframe.columns[index]
        col_name = col_name.replace("\n"," ")
        if "Unnamed" in col_name:
            corrected_col_names.append(corrected_col_names[index-1])
        else:
            corrected_col_names.append(col_name)
    dataframe.columns = corrected_col_names
    
    corrected_col_names = []
    for index in range(0,len(dataframe.columns)):
        col = dataframe.columns[index]
        first_row = dataframe.iloc[0,index]
        if (index > 5):
            new_column_name = col + " " + first_row 
            corrected_col_names.append(new_column_name)
        else:
            corrected_col_names.append(first_row)
    dataframe.columns = corrected_col_names
    # Note I had this as 3
    dataframe = dataframe.iloc[3:,]
    return dataframe

elections_2022_totals = clean_oh_import(elections_2022_totals)

In [18]:

elections_2022_totals.reset_index(inplace = True, drop = True)

In [19]:
elections_2022_totals

,County Name,Region Name,Media Market,Registered Voters,Ballots Counted,Official Voter Turnout,Governor and Lieutenant Governor Mike DeWine and Jon Husted (R),Governor and Lieutenant Governor Timothy Grady and Dayna Bickley (WI)*,Governor and Lieutenant Governor Craig Patton and Collin Cook (WI)*,Governor and Lieutenant Governor Renea Turner and Adina Pelletier (WI)*,...,Member of the State Board of Education - District 02 Teresa Fedor,Member of the State Board of Education - District 02 Sarah McGervey,Member of the State Board of Education - District 03 Charlotte McGuire,Member of the State Board of Education - District 04 Katie Hofmann,Member of the State Board of Education - District 04 Jenny Kilgore,Member of the State Board of Education - District 09 Robert R. Fulton,Member of the State Board of Education - District 09 John P. Hagan,Member of the State Board of Education - District 10 Tom Jackson,Member of the State Board of Education - District 10 Tim Miller,Member of the State Board of Education - District 10 Cierra Lynch Shehorn
0,Adams,Southwest,Cincinnati,17302,9038,0.522367,7348,1,0,3,...,0,0,0,0,0,0,0,0,0,0
1,Allen,West,Lima,66650,33031,0.495589,25461,7,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Ashland,Northeast,Cleveland,34699,18827,0.54258,14510,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,Ashtabula,Northeast,Cleveland,61091,31310,0.512514,20903,4,0,1,...,0,0,0,0,0,8647,16636,0,0,0
4,Athens,Southeast,Charleston,38989,19331,0.495807,8920,3,2,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Washington,Southeast,Parkersburg,42585,23075,0.541857,16934,4,0,2,...,0,0,0,0,0,0,0,0,0,0
84,Wayne,Northeast,Cleveland,73514,39126,0.532225,28284,4,0,2,...,0,0,0,0,0,0,0,0,0,0
85,Williams,Northwest,Toledo,25152,13147,0.522702,10331,0,1,4,...,0,0,0,0,0,0,0,0,0,0
86,Wood,Northwest,Toledo,93063,49132,0.527943,31399,7,3,1,...,22597,16341,0,0,0,0,0,0,0,0


In [20]:
races_to_clean = [i for i in elections_2022_totals.columns if i not in ['County Name', 'Precinct Name', 'Precinct Code', 'Region Name',
       'Media Market', 'Registered Voters', 'Ballots Counted',
       'Official Voter Turnout'] and "Member of the State Board of Education" not in i and "Judge of the Court of Appeals" not in i]

In [21]:
elections_2022_totals = elections_2022_totals[['County Name', 'Region Name',
       'Media Market', 'Registered Voters', 'Ballots Counted',
       'Official Voter Turnout'] + races_to_clean]

In [22]:
fips_file = pd.read_csv("./raw-from-source/FIPS/US_FIPS_Codes.csv")
fips_file = fips_file[fips_file["State"]=="Ohio"]
fips_file["FIPS County"] = fips_file["FIPS County"].astype(str)
fips_file["FIPS County"] = fips_file["FIPS County"].str.zfill(3)
fips_file["County Name"] = fips_file["County Name"].replace("De Kalb","DeKalb")
fips_dict = dict(zip(fips_file['County Name'], fips_file['FIPS County']))
elections_2022_totals['county_fips'] = elections_2022_totals['County Name'].map(fips_dict).fillna(elections_2022_totals['County Name'])
elections_2022_totals['county_fips'] = elections_2022_totals['county_fips'].astype(str)
elections_2022_totals['county_fips'] = elections_2022_totals['county_fips'].str.zfill(3)

# Print statements to check the county FIPs we've added
print(elections_2022_totals['county_fips'].unique())

['001' '003' '005' '007' '009' '011' '013' '015' '017' '019' '021' '023'
 '025' '027' '029' '031' '033' '035' '037' '039' '041' '043' '045' '047'
 '049' '051' '053' '055' '057' '059' '061' '063' '065' '067' '069' '071'
 '073' '075' '077' '079' '081' '083' '085' '087' '089' '091' '093' '095'
 '097' '099' '101' '103' '105' '107' '109' '111' '113' '115' '117' '119'
 '121' '123' '125' '127' '129' '131' '133' '135' '137' '139' '141' '143'
 '145' '147' '149' '151' '153' '155' '157' '159' '161' '163' '165' '167'
 '169' '171' '173' '175']


/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_20400/2351231091.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elections_2022_totals['county_fips'] = elections_2022_totals['County Name'].map(fips_dict).fillna(elections_2022_totals['County Name'])
/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_20400/2351231091.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elections_2022_totals['county_fips'] = elections_2022_totals['county_fips'].astype(str)
/var/folders/1t/0q4w6hm92mg_zxd8

In [23]:
races_to_clean

['Governor and Lieutenant Governor  Mike DeWine and Jon Husted (R)',
 'Governor and Lieutenant Governor  Timothy Grady and Dayna Bickley (WI)*',
 'Governor and Lieutenant Governor  Craig Patton and Collin Cook (WI)*',
 'Governor and Lieutenant Governor  Renea Turner and Adina Pelletier (WI)*',
 'Governor and Lieutenant Governor  Marshall Usher and Shannon  Walker (WI)*',
 'Governor and Lieutenant Governor  Nan Whaley and Cheryl L. Stephens (D)',
 'Attorney General  Jeffrey A. Crossman (D)',
 'Attorney General  Dave Yost (R)',
 'Auditor of State  Keith Faber (R)',
 'Auditor of State  Taylor Sappington (D)',
 'Secretary of State  Chelsea Clark (D)',
 'Secretary of State  Frank LaRose (R)',
 'Secretary of State  Terpsehore Tore Maras',
 'Treasurer of State  Scott Schertzer (D)',
 'Treasurer of State  Robert Sprague (R)',
 'Chief Justice of the Supreme Court Term Commencing 01/01/2023 Jennifer Brunner (D)',
 'Chief Justice of the Supreme Court Term Commencing 01/01/2023 Sharon L. Kennedy (

In [24]:
def get_race(race_string):
    race_string = race_string.title()
    race_string = race_string.replace("(Vote For 1)","")
    if "U.S. House" in race_string or 'Us House' in race_string or "Representative To Congress" in race_string:
        return "CON"
    elif "State House" in race_string or "State Representative" in race_string:
        return "SL"
    elif "State Senate" in race_string or "State Senator" in race_string:
        return "SU"
    elif "President" in race_string:
        return "PRE"
    elif "US Senate" in race_string or "Us Senate" in race_string or "U.S. Senator" in race_string:
        return "USS"
    elif "Public Service" in race_string:
        return "PSC"
    elif "Attorney General" in race_string:
        return "ATG"
    elif "Auditor General" in race_string or "Auditor" in race_string:
        return "AUD"
    elif "Treasurer" in race_string:
        return "TRE"
    elif "Superintendent" in race_string:
        return "SUP"
    elif "Secretary Of State" in race_string:
        return "SOS"
    elif "Governor" in race_string:
        return "GOV"
    elif "Commissioner Of Labor" in race_string:
        return "LAB"
    elif "Commissioner Of Agriculture" in race_string:
        return "AGR"
    elif "Commissioner Of Insurance" in race_string:
        return "AGR"
    elif "Chief Justice Of The Supreme Court" in race_string:
        return "CJU"
    elif "Justice Of The Supreme Court" in race_string:
        return "JUS"
    elif "Amendment" in race_string:
        if "1" in race_string:
            return "A01"
        elif "2" in race_string:
            return "A02"
        else:
            print("No race for:", race_string)
            raise ValueError
    elif "Referendum" in race_string:
        if "1" in race_string or "A" in race_string:
            return "RFA"
        elif "2" in race_string or "B" in race_string:
            return "RFB"
        else:
            print("No race for:", race_string)
            raise ValueError
    else:
        print("No race for:", race_string)
        raise ValueError
        
def get_election_type(race_string):
    if "(runoff)" in race_string:
        return "R"
    else:
        return "G"
        
def get_party(race_string):
    if "(R)" in race_string:
        return "R"
    elif "(D)" in race_string or "(Democrat)" in race_string:
        return "D"
    elif "(L)" in race_string or "(L)" in race_string:
        return "L"
    elif "(WI)" in race_string:
        return "O"
    elif race_string[0:3]=="Yes":
        return "YES"
    elif race_string[0:2]=="No":
        return "NO"
    else:
        return "O"
           
def get_name(name_string):
    if " (" in name_string and "Amendment" not in name_string and "Referendum" not in name_string:
        #print(name_string)
        name_string = name_string.split(" (")[0]
        name_string = name_string.replace("'","")
        likely_last = name_string.split(" ")[-1]
        proposed_last = likely_last[:3]
        if proposed_last in ['II', 'III', 'Jr', 'Jr.', 'Sr.', 'JR.', "JR", "IV"]:
            likely_last = name_string.split(" ")[-2]
            proposed_last = likely_last[:3]
        #print(proposed_last.upper())
        return proposed_last.upper()
    else:
        return name_string.split(" ")[-1][:3].upper()
#     name_string = name_string.split("-:-")[1]
#     name_string = name_string.replace(" (I)","")
#     name_string = name_string.replace("'","")
#     likely_last = name_string.split(" ")[-1]
#     proposed_last = likely_last[:3]
#     if proposed_last in ['II', 'III', 'Jr', 'Jr.', 'Sr.', 'JR.', "JR", "IV"]:
#         likely_last = name_string.split(" ")[-2]
#         proposed_last = likely_last[:3]
#     return proposed_last.upper()

def get_district(race_string, fill_level):
    race_string = race_string.replace(" (Vote For 1)","")
    break_word = "District "
    temp = race_string.split(break_word)[1].split(" ")[0]
    return temp.zfill(fill_level)

def column_rename_function(name_string):
    election_type = get_election_type(name_string)
    year = "22"
    party = get_party(name_string)
    race = get_race(name_string)
    district = ""
    if race in ["CON", "SU"]:
        district = get_district(name_string, 2)
        year = ""
    elif race in ["SL"]:
        district = get_district(name_string, 3)
        year = ""
    name = get_name(name_string)
    new_col_name = election_type + year + race + district + party + name
#     print("election_type: ", election_type)
#     print("year: ", year)
#     print("race: ", race)
#     print("district: ", district)
#     print("party: ", party)
#     print("name: ", name)
    if len(new_col_name) > 10:
        print(name_string)
        print(new_col_name)
    return new_col_name

# Make a dictionary that points to the new column names and checks for duplicates

update_dict = {'G22GOVRHUS':'G22GOVRDEW',
 'G22GOVDSTE':'G22GOVDWHA'}

race_updates_dict = {}
race_updates_reversed = {}
clean_dups = {}
new_names = []
for val in races_to_clean:
    new_name = column_rename_function(val)
    if new_name in update_dict.keys():
        new_name = update_dict[new_name]

    race_updates_dict[val] = new_name
    if new_name not in new_names:
        new_names.append(new_name)
        race_updates_reversed[new_name] = val
    else:
        print("Duplicate", new_name)
        print(race_updates_reversed[new_name])
        print(val)
        clean_dups[val] = race_updates_reversed[new_name]

In [25]:
race_updates_dict

{'Governor and Lieutenant Governor  Mike DeWine and Jon Husted (R)': 'G22GOVRDEW',
 'Governor and Lieutenant Governor  Timothy Grady and Dayna Bickley (WI)*': 'G22GOVOBIC',
 'Governor and Lieutenant Governor  Craig Patton and Collin Cook (WI)*': 'G22GOVOCOO',
 'Governor and Lieutenant Governor  Renea Turner and Adina Pelletier (WI)*': 'G22GOVOPEL',
 'Governor and Lieutenant Governor  Marshall Usher and Shannon  Walker (WI)*': 'G22GOVOWAL',
 'Governor and Lieutenant Governor  Nan Whaley and Cheryl L. Stephens (D)': 'G22GOVDWHA',
 'Attorney General  Jeffrey A. Crossman (D)': 'G22ATGDCRO',
 'Attorney General  Dave Yost (R)': 'G22ATGRYOS',
 'Auditor of State  Keith Faber (R)': 'G22AUDRFAB',
 'Auditor of State  Taylor Sappington (D)': 'G22AUDDSAP',
 'Secretary of State  Chelsea Clark (D)': 'G22SOSDCLA',
 'Secretary of State  Frank LaRose (R)': 'G22SOSRLAR',
 'Secretary of State  Terpsehore Tore Maras': 'G22SOSOMAR',
 'Treasurer of State  Scott Schertzer (D)': 'G22TREDSCH',
 'Treasurer of St

In [26]:
elections_2022_totals.rename(columns = race_updates_dict, inplace = True)

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_20400/1695302052.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elections_2022_totals.rename(columns = race_updates_dict, inplace = True)


## Drop Columns with Zero Votes

In [27]:
zero_votes = []
for val in race_updates_dict.values():
    official = sum(elections_2022_totals[val])
    rdh = sum(elections_2022[val])
    if rdh == 0:
        zero_votes.append(val)

In [28]:
elections_2022.drop(zero_votes, axis = 1, inplace = True)

In [29]:
check_cols = [i for i in race_updates_dict.values() if i not in zero_votes]

In [30]:
check_cols_dict = {k:v for k,v in race_updates_dict.items() if v not in zero_votes}

In [31]:
pd.DataFrame(check_cols_dict.items()).to_csv("./field_names.csv", index = False)

## Statewide Totals

In [32]:
for val in check_cols:
    official = sum(elections_2022_totals[val])
    rdh = sum(elections_2022[val])
    if official!= rdh:
        print(val)
        print("\tOfficial", official)
        print("\tRDH", rdh)

## Countywide Totals

In [33]:
pdv.county_totals_check(elections_2022_totals, "County", elections_2022, "RDH", check_cols, "county_fips", full_print=False, method='county')

***Countywide Totals Check***

Counties that match:

['001', '003', '005', '007', '009', '011', '013', '015', '017', '019', '021', '023', '025', '027', '029', '031', '033', '035', '037', '039', '041', '043', '045', '047', '049', '051', '053', '055', '057', '059', '061', '063', '065', '067', '069', '071', '073', '075', '077', '079', '081', '083', '085', '087', '089', '091', '093', '095', '097', '099', '101', '103', '105', '107', '109', '111', '113', '115', '117', '119', '121', '123', '125', '127', '129', '131', '133', '135', '137', '139', '141', '143', '145', '147', '149', '151', '153', '155', '157', '159', '161', '163', '165', '167', '169', '171', '173', '175']


## Export File

In [34]:
elections_2022["UNIQUE_ID"] = elections_2022["County Name"] + "-" + elections_2022["Precinct Name"]

In [35]:
elections_2022.rename(columns = {"county_fips":"COUNTYFP"}, inplace = True)

In [36]:
list(elections_2022.columns[8:-2])

['G22GOVRDEW',
 'G22GOVDWHA',
 'G22ATGDCRO',
 'G22ATGRYOS',
 'G22AUDRFAB',
 'G22AUDDSAP',
 'G22SOSDCLA',
 'G22SOSRLAR',
 'G22SOSOMAR',
 'G22TREDSCH',
 'G22TRERSPR',
 'G22CJUDBRU',
 'G22CJURKEN',
 'G22JUSRFIS',
 'G22JUSDJAM',
 'G22JUSRDEW',
 'G22JUSDZAY',
 'G22USSDRYA',
 'G22USSRVAN',
 'GCON01RCHA',
 'GCON01DLAN',
 'GCON02DMEA',
 'GCON02RWEN',
 'GCON03DBEA',
 'GCON03RSTA',
 'GCON04RJOR',
 'GCON04DWIL',
 'GCON05RLAT',
 'GCON05DSWA',
 'GCON06RJOH',
 'GCON06DLYR',
 'GCON07DDIE',
 'GCON07RMIL',
 'GCON08RDAV',
 'GCON08DENO',
 'GCON09DKAP',
 'GCON09RMAJ',
 'GCON10DESR',
 'GCON10RTUR',
 'GCON11RBRE',
 'GCON11DBRO',
 'GCON12RBAL',
 'GCON12DRIP',
 'GCON13RGIL',
 'GCON13DSYK',
 'GCON14RJOY',
 'GCON14DKIL',
 'GCON15RCAR',
 'GCON15DJOS',
 'GSU01RMCC',
 'GSU03DMAH',
 'GSU03RREY',
 'GSU05RHUF',
 'GSU07DDAL',
 'GSU07RWIL',
 'GSU09DING',
 'GSU09RSON',
 'GSU11RDIA',
 'GSU11DHIC',
 'GSU13DELI',
 'GSU13RMAN',
 'GSU15DCRA',
 'GSU17DBOO',
 'GSU17RWIL',
 'GSU19RBRE',
 'GSU19DSWI',
 'GSU21RALT',
 'GSU21DSMI',

In [37]:
race_cols = list(elections_2022.columns[8:-2])
race_cols.sort()

In [38]:
elections_2022 = elections_2022[["UNIQUE_ID", "COUNTYFP", 'County Name', 'Precinct Name', 'Precinct Code', 'Region Name',
       'Media Market', 'Registered Voters', 'Ballots Counted',
       'Official Voter Turnout']+ race_cols]

In [39]:
[i for i in elections_2022.columns if len(i)>10]

['County Name',
 'Precinct Name',
 'Precinct Code',
 'Region Name',
 'Media Market',
 'Registered Voters',
 'Ballots Counted',
 'Official Voter Turnout']

In [40]:
final_updates = {'County Name':'County',
 'Precinct Name':'PRECNAME',
 'Precinct Code':'PRECCODE',
 'Region Name':'Region',
 'Media Market':'Market',
 'Registered Voters':'REGVOT',
 'Ballots Counted':'BALLOTS',
 'Official Voter Turnout':'TURNOUT'}

In [41]:
elections_2022.rename(columns = final_updates, inplace = True)

## Export File

In [42]:
elections_2022.to_csv("./oh_2022_gen_prec.csv", index = False)